In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

from IPython.display import Audio

def meow():
    display(Audio(filename='../cat_meow2.wav', autoplay=True))

In [2]:
meow()

In [17]:
from epsampling.modeling import get_full_modeling_dfs

df, df_acs = get_full_modeling_dfs(timestamp='20241009-144131')

df = df[df.True_inc_deaths > 0]
df

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,...,HU_OCC,POP_NH,HINS_A0018,HINS_A65p,HH_x5,POP_A3564,HU_x14,POP_16p_EMP_x7,POP_A25p,POP_x2
0,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,...,0.793727,0.481868,0.794219,0.318318,-0.147691,-0.134211,-0.155939,-0.148767,-0.141173,-0.140801
1,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,...,0.793727,0.481868,0.794219,0.318318,-0.147691,-0.134211,-0.155939,-0.148767,-0.141173,-0.140801
2,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,...,0.793727,0.481868,0.794219,0.318318,-0.147691,-0.134211,-0.155939,-0.148767,-0.141173,-0.140801
3,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,...,0.793727,0.481868,0.794219,0.318318,-0.147691,-0.134211,-0.155939,-0.148767,-0.141173,-0.140801
4,1001,1,58239.0,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,...,0.793727,0.481868,0.794219,0.318318,-0.147691,-0.134211,-0.155939,-0.148767,-0.141173,-0.140801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304392,56045,56,6891.0,576641,0.011950,20211120,42.000000,13.0,10.0,3.0,...,-0.151876,0.566402,-1.771829,0.515204,-0.306096,-0.288444,-0.313782,-0.287296,-0.292354,-0.293789
304394,56045,56,6891.0,576641,0.011950,20211204,26.000000,14.0,13.0,1.0,...,-0.151876,0.566402,-1.771829,0.515204,-0.306096,-0.288444,-0.313782,-0.287296,-0.292354,-0.293789
304399,56045,56,6891.0,576641,0.011950,20220108,19.000000,16.0,14.0,2.0,...,-0.151876,0.566402,-1.771829,0.515204,-0.306096,-0.288444,-0.313782,-0.287296,-0.292354,-0.293789
304403,56045,56,6891.0,576641,0.011950,20220205,23.000000,17.0,16.0,1.0,...,-0.151876,0.566402,-1.771829,0.515204,-0.306096,-0.288444,-0.313782,-0.287296,-0.292354,-0.293789


In [18]:
df['Target'] = df.apply(lambda x: (x.True_inc_deaths+1) / x.Pop * 100000, axis=1)
df['Naive'] = df.apply(lambda x: (x.Naive_inc_deaths+1) / x.Pop * 100000, axis=1)
df['State'] = df.apply(lambda x: (x.Proj_inc_deaths+1) / x.State_pop * 100000, axis=1)
df['Cumtm1'] = df.apply(lambda x: (x.Cum_deaths_tm1+1) / x.Pop * 100000, axis=1)
# df['Target'].describe()

In [19]:
# # # # # # # # # #
how_many = 8
# # # # # # # # # #

In [20]:
from epsampling.utils import get_chunks
from epsampling.modeling import get_date_chunked_splits, get_performance
from sklearn import linear_model

chunks = get_chunks(list(df.Date.unique()), 4)
idc = [12]

for idx in idc:
    
    df_train, df_test = get_date_chunked_splits(df, chunks, idx)


    feats = list(df_acs.columns) + ['State','Cumtm1','Naive',
                                   'State_pop','Ratio','Pop']
    
    df_train, df_test = get_date_chunked_splits(df, chunks, idx)
    
    X_train = df_train[feats]
    X_test = df_test[feats]

    y_train = df_train['Target']
    y_test = df_test['Target']
    y_naive = df_test['Naive']
    
    reg = linear_model.Lasso(alpha=0.1)
    reg.fit(X_train, y_train)
    
    y_pred = reg.predict(X_test)

In [21]:
coef = abs(reg.coef_)
# print(coef, abs(coef))
# print(coef)

# print(sorted(coef, reverse=True))
ind = np.argpartition(coef, -how_many)[-how_many:]
top = coef[ind]

# print(ind,top)
df_coefs = X_train.iloc[:, ind]
which_feats = df_coefs.columns


# xticks = [acs_dict[x] for x in res_df['feat_name'].values]
# print(xticks)

acs_dict = {'EDU_DOCT':'PhD pop',
            'HHI_125_150k': 'Income 125-150k',
            'HHI_10_15k': 'Income 10-15k',
            'HINS_A3564': 'Health ins\n(age 35-64)',
            'HHI_150_200k': 'Income 150-200k',
            'Naive': 'Naive',
            'POP_A85p': 'Pop age >= 85',
            'EDU_BACH': 'BS pop',
            'State':'State',
            
           }

which_featss = [acs_dict[x] for x in which_feats]

res_df = pd.DataFrame([top, which_featss], index=['coef','feat_name'])
res_df = res_df.T.sort_values(by='coef',inplace=False,ascending=False)
res_df = res_df[:how_many]

KeyError: 'EDU_MAST'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# xticks = [acs_dict[x] for x in res_df['feat_name'].values]
# print(xticks)

DPI = 300

plt.figure(figsize=(10,5))
sns.barplot(res_df, y="coef", x="feat_name", legend=False)
plt.title('Lasso, alpha = 0.1')
# plt.xticks(xticks, rotation=45)
plt.xticks(rotation=45,ha='right',wrap=True)

sns.despine()
plt.gcf().set_dpi(DPI)
plt.show()

In [ ]:
y_pred = reg.predict(X_test)
y_naive = df_test['Naive']

model_names = ['Deaths_pred', 'Deaths_naive']
model_preds = [y_pred, y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)

for k,v in metrics_dict.items():
    print(f'\n\n* {k} *\n')
    for k2,v2 in v.items():
        if k2=='r2':
            k3 = 'r-squared'
        else:
            k3 = k2
        print(f' • {k3}: \t{round(v2,5)}')